# Accessing data from Google Cloud Storage

### Purpose
The purpose of this notebook is to explore using Google Cloud Storage to cache covid data to reduce the need to request and process the data each time the heroku server is accessed.

- **First**, connect and read from cloud storage, from a notebook.
- **Second**, write to cloud to storage, from a notebook.
- **Third**, run the server locally and test reading from cloud storage.
- **Fourth**, update the heroku server and test reading from cloud storage.
- **Fifth**, write a script that will request and process data from the source and write to cloud storage.
- **Sixth**, deploy an app-engine instance that can update the bucket as needed.
- **Seventh**, create a cron job on GCP to trigger the app-engine instance every morning to update retrieve, process, and update data.

---

## 1 - Notebook Setup

First we need to knock this notebook one directory up to the root directory of the app.

This is just for cleanliness purposes; to keep all notebooks in a notebooks directory.

In [2]:
pwd

'/Users/DanOvadia/Projects/covid-hotspots/notebooks'

In [3]:
cd ..

/Users/DanOvadia/Projects/covid-hotspots


## Import Libraries

In [4]:
import os
import sys
import time
import requests

import pandas as pd
import gzip
from io import BytesIO, TextIOWrapper
from google.cloud import storage

from config import config
from modules import data_processing

%load_ext autoreload
%autoreload 1
%aimport config.config
%aimport modules.data_processing

In order to access google cloud storage we need to set an environment variable `GOOGLE_APPLICATION_CREDENTIALS` for this session. This environment variable will point towards the service account credentials json.

In the actual app, we will handle this separately.

In [5]:
# App Engine default service account credentials 
config_file = config.config.service_account_credentials_file

In [6]:
# Assign enviroment variable if it hasn't been assigned yet
if 'GOOGLE_APPLICATION_CREDENTIALS' not in os.environ:
    %env GOOGLE_APPLICATION_CREDENTIALS=/Users/DanOvadia/Projects/covid-hotspots/config/$config_file
    
print(f"Credentials already in environment: {'GOOGLE_APPLICATION_CREDENTIALS' in os.environ}")
print(f"export GOOGLE_APPLICATION_CREDENTIALS=/Users/DanOvadia/Projects/covid-hotspots/config/{config_file}")

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/DanOvadia/Projects/covid-hotspots/config/ytd-shared-project-eb630837f7b3.json
Credentials already in environment: True
export GOOGLE_APPLICATION_CREDENTIALS=/Users/DanOvadia/Projects/covid-hotspots/config/ytd-shared-project-eb630837f7b3.json


Lets test to see if we've established our authentication for google cloud storage.

### List available buckets for our service account

In [ ]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = [bucket.name for bucket in storage_client.list_buckets()]
    print(buckets)
    return buckets
a = implicit()

Yup, we look good. Now, lets import the data.

## Connect to cloud storage

First we create a `google.cloud.storage.client.Client` object: Client to bundle configuration needed for API requests.

In [ ]:
CLIENT = storage.Client()

Using the name of a bucket, we grab a bucket object from our client bundle.

In [ ]:
BUCKET_NAME = 'us_covid_hotspot-bucket'
BUCKET = CLIENT.get_bucket(bucket_or_name = BUCKET_NAME)

In [ ]:
[blob.name for blob in BUCKET.list_blobs()]

In [ ]:
FILENAME = 'covid_states.csv'

In [ ]:
blob = storage.blob.Blob(name = BLOB_NAME, bucket = BUCKET_NAME)

In [ ]:
BUCKET.get_blob()

In [ ]:
BUCKET_NAME = 'us_covid_hotspot-bucket'
BUCKET = client.get_bucket(bucket_or_name=BUCKET_NAME)

In [ ]:
BUCKET_NAME = 'us_covid_hotspot-bucket'
BLOB_NAME = "covid_states.csv"

In [ ]:
df = get_df_from_blob(BUCKET_NAME, BLOB_NAME)

In [ ]:
def get_df_from_blob(bucket_name, blob_name):
    client = storage.Client()
    
    #bucket = client.get_bucket(bucket_or_name = bucket_name)
    #blob = bucket.get_blob(blob_name=blob_name)
    
    BLOB_URI = f"gs://{BUCKET_NAME}/{BLOB_NAME}"
    
    return pd.read_csv(BLOB_URI, compression = 'gzip')

## Reading from a blob

I attempted two ways to read from the blob. Converting the blob to a string, storing the string, then using BytesIO to read that string and send it to pd.read_CSV.

I also attempted a method provided by pandas to read direction from a blob using the `gs://[BUCKET_NAME]/[BLOB_NAME]` URI of the blob.

In [ ]:
BLOB_URI = f"gs://{BUCKET_NAME}/{BLOB_NAME}"

In [ ]:
BLOB_URI

In [ ]:
%%time
df = pd.read_csv(BLOB_URI)

In [ ]:
df.head()

In [ ]:
%%time
content = blob.download_as_string()
df = pd.read_csv(BytesIO(content))

## Writing to Blob

In [ ]:
BUCKET_NAME = 'us_covid_hotspot-bucket'
#BLOB_NAME = "covid_states.csv"
BLOB_NAME = "covid_states_20200901.csv.gz"
FILE_PATH = f"data/{BLOB_NAME}"

In [ ]:
%%time
write_blob_to_gcs(BUCKET_NAME, 
                  BLOB_NAME, 
                  FILE_PATH)

Now lets test to see if we got the result we wanted

In [ ]:
%%time 
df = get_df_from_blob(BUCKET_NAME, BLOB_NAME)

In [ ]:
df.head()

In [ ]:
def write_blob_to_gcs(bucket_name, blob_name, filepath):
    # Client to bundle configuration needed for API requests.
    client = storage.Client()

    # Extract the bucket object from the client bundle
    bucket = client.get_bucket(bucket_name)

    # Instantiate or extract the blob object from the bucket
    blob = storage.blob.Blob(blob_name,bucket)

    # Upload the file to the specific blob
    blob.upload_from_filename(filepath)
    
    return [blob_item.name for blob_item in bucket.list_blobs()]

## Reading from Blob

In [ ]:
BUCKET_NAME = 'us_covid_hotspot-bucket'
BLOB_NAME = 

In [ ]:
write_blob_to_gcs()

In [ ]:
[blob_item.name for blob_item in BUCKET.list_blobs()]

### Trying to zip and save

In [ ]:
covid_counties.head()

In [ ]:
# write a pandas dataframe to gzipped CSV file
covid_counties.to_csv("data/education_salary.csv.gz", 
           index=False, 
           compression="gzip")

In [ ]:
pd.read_csv()

In [ ]:
a = pd.read_csv("data/covid_states_20200901.csv.gz", compression="gzip")

In [ ]:
df = pd.read_csv('gs://us_covid_hotspot-bucket/covid_states_20200901.csv.gz')

In [ ]:
import time

today = time.strftime('%Y%m%d')
filepath = f'data/covid_states_{today}.csv.gz'
cache_mode = 2

(os.path.exists(filepath) and cache_mode in (1,2))

In [ ]:
#a = data_processing.get_covid_county_data()
b = data_processing.get_covid_state_data(cache_mode = 3)


In [ ]:
b.head()

In [ ]:
%%time
bucket_name = 'us_covid_hotspot-bucket'
blob_name = "covid_counties_20200901.csv.gz"
blob_uri = f"gs://{bucket_name}/{blob_name}"
print(f"Pulling county data from GCS [{blob_uri}]")

# Get the client object to make the request
client = storage.Client()

df = pd.read_csv(blob_uri, compression = 'gzip')

df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

In [ ]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

### Using gzip and IO

Since we Cloud Function is serverless, we cannot temporarily store `covid_states.csv.gz` so we'll need to store it in memory as a `BytesIO` object in buffer.

In [ ]:
import gzip
import pandas as pd
from io import BytesIO, TextIOWrapper

In [ ]:
# Read in new county data from Covid Tracking Project
COVID_STATES_DF = data_processing.get_covid_state_data(cache_mode = 0)

In [ ]:
BUCKET_NAME = 'us_covid_hotspot-bucket'
BLOB_NAME = 'covid_states2.csv.gz'
FILEPATH = f"data/{BLOB_NAME}"
BLOB_URI = f"gs://{BUCKET_NAME}/{BLOB_NAME}"

In [ ]:
type(gz_buffer)

In [ ]:
gz_buffer = BytesIO()

with gzip.GzipFile(mode='w',fileobj=gz_buffer) as gz_file:
    COVID_STATES_DF.to_csv(TextIOWrapper(gz_file,'utf8'),
                          index=False)
gz_buffer.seek(0);

In [ ]:
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)

blob = storage.blob.Blob(BLOB_NAME, bucket)

In [ ]:
%%time
blob.upload_from_file(file_obj=gz_buffer, content_type = 'text/csv')

In [ ]:
df = pd.read_csv(BLOB_URI)

In [ ]:
BLOB_NAME = 'covid_counties.csv.gz'

In [8]:
COVID_COUNTIES_DF.head()

NameError: name 'COVID_COUNTIES_DF' is not defined

In [7]:
COVID_COUNTIES_DF = data_processing.get_covid_county_data(cache_mode = 0)

Retrieving Covid County data
Pulling county data from github.


TypeError: incompatible index of inserted column with frame index

In [ ]:
a = data_processing.get_census_county_data()

In [ ]:
def write_df_to_GCS(df, blob_name):
    bucket_name = 'us_covid_hotspot-bucket'
    blob_uri = f"gs://{bucket_name}/{blob_name}"
    
    # Instantiate BytesIO Object
    gz_buffer = BytesIO()
    
    # Instantiate a GzipFile Object using the BytesIO object, 
      # write to it using a text wrapper.
    with gzip.GzipFile(mode='w', fileobj=gz_buffer) as gz_file:
        df.to_csv(TextIOWrapper(gz_file,'utf8'),
                 index=False)
    gz_buffer.seek(0)
    
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = storage.blob.Blob(blob_name, bucket)
    
    blob.upload_from_file(file_obj=gz_buffer, content_type='text/csv')
    return [blob_item.name for blob_item in bucket.list_blobs()]

write_df_to_GCS(COVID_COUNTIES_DF,BLOB_NAME)